In [78]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [79]:
import pandas as pd

In [80]:
import lxml.etree as ET

In [4]:
import os

In [81]:
path = r'/content/gdrive/MyDrive/Colab Notebooks/bdpdata'
files = os.listdir(path)

In [82]:
files_xml = [x for x in files if x[-3:] == 'xml']

In [84]:
nct_id = []
brief_title = []
brief_summary = []
detailed_description = []
overall_status = []
phase = []
study_type = []
primary_purpose = []
criteria = []
gender = []
min_age = []
max_age = []
intervention = []
for f in files_xml:
    fullname = os.path.join(path, f)
    tree = ET.parse(fullname)
    ntc_root = tree.getroot()
    for i in ntc_root.findall('id_info'):
        id = i.find('nct_id').text
        nct_id.append(id)
        
    for i in ntc_root.findall('brief_title'):
        title = i.text
        brief_title.append(title)
    
    for i in ntc_root.findall('brief_summary'):
        summary = i.find('textblock').text
        brief_summary.append(summary)
    
    for i in ntc_root.findall('detailed_description'):
        description = i.find('textblock').text
        detailed_description.append(description)
    
    for i in ntc_root.findall('overall_status'):
        status = i.text
        overall_status.append(status)
    
    for i in ntc_root.findall('phase'):
        p = i.text
        phase.append(p)

    for i in ntc_root.findall('study_type'):
        type = i.text
        study_type.append(type)

    for i in ntc_root.findall('study_design_info'):
        if i.find('primary_purpose') == None:
            primary_purpose.append('N/A')
        else:
            purpose = i.find('primary_purpose').text
            primary_purpose.append(purpose)

    for i in ntc_root.findall('eligibility'):
        for j in i.findall('criteria'):
            crit = j.find('textblock').text
            criteria.append(crit)

    for i in ntc_root.findall('eligibility'):
        gen = i.find('gender').text
        gender.append(gen)

    for i in ntc_root.findall('eligibility'):
        min_a = i.find('minimum_age').text
        min_age.append(min_a)
    
    for i in ntc_root.findall('eligibility'):
        max_a = i.find('maximum_age').text
        max_age.append(max_a)
        
    for i in ntc_root.findall('intervention_browse'):
        interven_2 = i.find('mesh_term').text
        intervention.append(interven_2)

In [85]:
datas=[nct_id,brief_title,brief_summary,detailed_description,overall_status,phase,study_type,
      criteria,gender,min_age,max_age,intervention]
ntc_1 = pd.DataFrame(datas).T
ntc_1 = ntc_1.rename(columns={0:'nct_id',1:'brief_title',2:'brief_summary',3:'detailed_description',
                              4:'overall_status',5:'phase',6:'study_type',7:'criteria',8:'gender',
                              9:'min_age',10:'max_age',11:'intervention'})

In [14]:
ntc_1_duplicate = ntc_1

In [15]:
ntc_1.shape

(510, 12)

In [16]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
from nltk.tokenize import word_tokenize

In [18]:
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
def preprocess(data):
  import re
  pattern = "[^a-zA-Z]"
  html_pattern = re.compile('<.*?>')
  #stemmer = PorterStemmer()
  #lemmatizer = WordNetLemmatizer()
  new = []
  for txt in data:
    txt = txt.lower()
    txt = re.sub(pattern, " ", txt)
    txt = re.sub(r"http\S+", "", txt)
    #txt = re.sub(html_pattern, "", txt)
    txt = word_tokenize(txt)
    #txt = re.sub(html_pattern, "", txt)
    #txt = [stemmer.stem(word) for word in txt]
    #txt = [lemmatizer.lemmatize(word) for word in txt]
    
    txt = " ".join(txt)
    new.append(txt)
  return new

In [20]:
ntc_1['brief_title'] = preprocess(ntc_1['brief_title'])
ntc_1['brief_summary'] = preprocess(ntc_1['brief_summary'])


In [21]:
ntc_1

,nct_id,brief_title,brief_summary,detailed_description,overall_status,phase,study_type,criteria,gender,min_age,max_age,intervention
0,NCT00000279,novel medications for opiate detoxification,the purpose of this study is to evaluate novel...,\n The purpose of this study was to deter...,Completed,Phase 2,Interventional,\n Please contact site for information....,All,18 Years,N/A,Clonidine
1,NCT00000277,mazindol for cocaine abuse,the purpose of this study is to evaluate high ...,\n The purpose of this study was to deter...,Completed,Phase 2,Interventional,\n Please contact site for information....,All,18 Years,N/A,Mazindol
2,NCT00000284,role of metabolites in nicotine dependence,the purpose of this study is to determine the ...,\n This study evaluated the effect of nef...,Completed,Phase 2,Interventional,\n Inclusion Criteria:\r\n\r\n M...,All,21 Years,45 Years,Amantadine
3,NCT00000281,pharmacotherapy for schizophrenic drug users,the purpose of this study is to evaluate maint...,"\n This 27-week double-blind, placebo con...",Completed,Phase 2,Interventional,\n Please contact site for information....,All,18 Years,55 Years,Phenytoin
4,NCT00000285,effects of phenytoin on cocaine use in humans,the purpose of this study is to determine the ...,"\n 1-Year study, participant will be rand...",Completed,Phase 1,Interventional,\n Inclusion Criteria:\r\n\r\n M...,All,20 Years,45 Years,Nimodipine
...,...,...,...,...,...,...,...,...,...,...,...,...
505,NCT00000597,multi center trial of anti thymocyte globulin ...,to determine the therapeutic effects of anti t...,None,Completed,None,Interventional,"\n Males and females, 5 years or older,...",All,18 Years,75 Years,None
506,NCT00000595,evaluation of subcutaneous desferrioxamine as ...,to determine whether deferoxamine prevented th...,None,Completed,None,Interventional,\n HIV-infected patients with CD4 count...,All,5 Years,N/A,None
507,NCT00000593,viral activation transfusion study vats,the purpose of the trial was to determine if t...,None,Completed,None,Interventional,\n Men and women with angiographically ...,All,18 Years,75 Years,None
508,NCT00000594,nhlbi type ii coronary intervention study,to determine whether lowering of cholesterol w...,None,Completed,None,Interventional,\n No eligibility criteria\r\n,All,21 Years,55 Years,None


In [22]:
ntc_1 = ntc_1.fillna(value = 'None')

In [23]:
ntc_1['combined']= ntc_1['brief_title'] + ntc_1['brief_summary'] + ntc_1['criteria'] + ntc_1['detailed_description'] + ntc_1['intervention'] + ntc_1['max_age'] + ntc_1['min_age'] + ntc_1['gender'] 

In [ ]:
ntc_1['combined'] = preprocess(ntc_1['combined'])

In [24]:
ntc_1['combined'].tail()

505    multi center trial of anti thymocyte globulin ...
506    evaluation of subcutaneous desferrioxamine as ...
507    viral activation transfusion study vatsthe pur...
508    nhlbi type ii coronary intervention studyto de...
509    diffuse fibrotic lung diseaseto determine the ...
Name: combined, dtype: object

In [25]:
ntc_1

,nct_id,brief_title,brief_summary,detailed_description,overall_status,phase,study_type,criteria,gender,min_age,max_age,intervention,combined
0,NCT00000279,novel medications for opiate detoxification,the purpose of this study is to evaluate novel...,\n The purpose of this study was to deter...,Completed,Phase 2,Interventional,\n Please contact site for information....,All,18 Years,N/A,Clonidine,novel medications for opiate detoxificationthe...
1,NCT00000277,mazindol for cocaine abuse,the purpose of this study is to evaluate high ...,\n The purpose of this study was to deter...,Completed,Phase 2,Interventional,\n Please contact site for information....,All,18 Years,N/A,Mazindol,mazindol for cocaine abusethe purpose of this ...
2,NCT00000284,role of metabolites in nicotine dependence,the purpose of this study is to determine the ...,\n This study evaluated the effect of nef...,Completed,Phase 2,Interventional,\n Inclusion Criteria:\r\n\r\n M...,All,21 Years,45 Years,Amantadine,role of metabolites in nicotine dependencethe ...
3,NCT00000281,pharmacotherapy for schizophrenic drug users,the purpose of this study is to evaluate maint...,"\n This 27-week double-blind, placebo con...",Completed,Phase 2,Interventional,\n Please contact site for information....,All,18 Years,55 Years,Phenytoin,pharmacotherapy for schizophrenic drug usersth...
4,NCT00000285,effects of phenytoin on cocaine use in humans,the purpose of this study is to determine the ...,"\n 1-Year study, participant will be rand...",Completed,Phase 1,Interventional,\n Inclusion Criteria:\r\n\r\n M...,All,20 Years,45 Years,Nimodipine,effects of phenytoin on cocaine use in humanst...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,NCT00000597,multi center trial of anti thymocyte globulin ...,to determine the therapeutic effects of anti t...,None,Completed,None,Interventional,"\n Males and females, 5 years or older,...",All,18 Years,75 Years,None,multi center trial of anti thymocyte globulin ...
506,NCT00000595,evaluation of subcutaneous desferrioxamine as ...,to determine whether deferoxamine prevented th...,None,Completed,None,Interventional,\n HIV-infected patients with CD4 count...,All,5 Years,N/A,None,evaluation of subcutaneous desferrioxamine as ...
507,NCT00000593,viral activation transfusion study vats,the purpose of the trial was to determine if t...,None,Completed,None,Interventional,\n Men and women with angiographically ...,All,18 Years,75 Years,None,viral activation transfusion study vatsthe pur...
508,NCT00000594,nhlbi type ii coronary intervention study,to determine whether lowering of cholesterol w...,None,Completed,None,Interventional,\n No eligibility criteria\r\n,All,21 Years,55 Years,None,nhlbi type ii coronary intervention studyto de...


In [ ]:
ntc_1

,nct_id,brief_title,brief_summary,detailed_description,overall_status,phase,study_type,criteria,gender,min_age,max_age,intervention,combined
0,NCT00000134,studies of the ocular complications of aids so...,to compare the relative merits of three therap...,\n CMV retinitis is the most common intra...,Completed,Phase 3,Interventional,\n inclusion criteria: Males and female...,All,18 Years,N/A,Ganciclovir,studies of the ocular complications of aids so...
1,NCT00000133,cryotherapy for retinopathy of prematurity cry...,to determine the safety and efficacy of trans ...,\n ROP is a disease of the eyes of premat...,Completed,N/A,Interventional,\n Premature infants of either gender w...,All,N/A,1 Year,Betaxolol,cryotherapy for retinopathy of prematurity cry...
2,NCT00000132,early manifest glaucoma trial emgt,the primary purpose is to compare the effect o...,\n Glaucoma is a common disease in older ...,Unknown status,Phase 3,Interventional,\n Men and women between ages 50 and 80...,All,50 Years,80 Years,Anti-Bacterial Agents,early manifest glaucoma trial emgtthe primary ...
3,NCT00000131,central vein occlusion study cvos,to determine whether photocoagulation therapy ...,\n Central vein occlusion is a common ret...,Completed,Phase 3,Interventional,\n Men and women must have been age 21 ...,All,21 Years,N/A,Antihypertensive Agents,central vein occlusion study cvosto determine ...
4,NCT00000130,endophthalmitis vitrectomy study evs,to determine the role of initial pars plana vi...,\n Endophthalmitis is a serious ocular in...,Completed,Phase 3,Interventional,\n Men and women were eligible for entr...,All,N/A,N/A,Polymethyl Methacrylate,endophthalmitis vitrectomy study evsto determi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,NCT00000647,an open trial combining zidovudine interferon ...,to restore immunologic function and virus free...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,18 Years,60 Years,None,an open trial combining zidovudine interferon ...
534,NCT00000648,a pilot study evaluating penicillin g and ceft...,to provide information on the response of hiv ...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,13 Years,N/A,None,a pilot study evaluating penicillin g and ceft...
535,NCT00000646,pentoxifylline trental as a modulator of tumor...,to determine whether pentoxifylline lowers tum...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,18 Years,N/A,None,pentoxifylline trental as a modulator of tumor...
536,NCT00000649,an open label staggered rising dose cohort stu...,to assess the safety and tolerance of multiple...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,18 Years,N/A,None,an open label staggered rising dose cohort stu...


In [ ]:
ntc_1

,nct_id,brief_title,brief_summary,detailed_description,overall_status,phase,study_type,criteria,gender,min_age,max_age,intervention,combined
0,NCT00000134,studies of the ocular complications of aids so...,to compare the relative merits of three therap...,\n CMV retinitis is the most common intra...,Completed,Phase 3,Interventional,\n inclusion criteria: Males and female...,All,18 Years,N/A,Ganciclovir,studies of the ocular complications of aids so...
1,NCT00000133,cryotherapy for retinopathy of prematurity cry...,to determine the safety and efficacy of trans ...,\n ROP is a disease of the eyes of premat...,Completed,N/A,Interventional,\n Premature infants of either gender w...,All,N/A,1 Year,Betaxolol,cryotherapy for retinopathy of prematurity cry...
2,NCT00000132,early manifest glaucoma trial emgt,the primary purpose is to compare the effect o...,\n Glaucoma is a common disease in older ...,Unknown status,Phase 3,Interventional,\n Men and women between ages 50 and 80...,All,50 Years,80 Years,Anti-Bacterial Agents,early manifest glaucoma trial emgtthe primary ...
3,NCT00000131,central vein occlusion study cvos,to determine whether photocoagulation therapy ...,\n Central vein occlusion is a common ret...,Completed,Phase 3,Interventional,\n Men and women must have been age 21 ...,All,21 Years,N/A,Antihypertensive Agents,central vein occlusion study cvosto determine ...
4,NCT00000130,endophthalmitis vitrectomy study evs,to determine the role of initial pars plana vi...,\n Endophthalmitis is a serious ocular in...,Completed,Phase 3,Interventional,\n Men and women were eligible for entr...,All,N/A,N/A,Polymethyl Methacrylate,endophthalmitis vitrectomy study evsto determi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,NCT00000647,an open trial combining zidovudine interferon ...,to restore immunologic function and virus free...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,18 Years,60 Years,None,an open trial combining zidovudine interferon ...
534,NCT00000648,a pilot study evaluating penicillin g and ceft...,to provide information on the response of hiv ...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,13 Years,N/A,None,a pilot study evaluating penicillin g and ceft...
535,NCT00000646,pentoxifylline trental as a modulator of tumor...,to determine whether pentoxifylline lowers tum...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,18 Years,N/A,None,pentoxifylline trental as a modulator of tumor...
536,NCT00000649,an open label staggered rising dose cohort stu...,to assess the safety and tolerance of multiple...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,18 Years,N/A,None,an open label staggered rising dose cohort stu...


In [ ]:
ntc_1.tail()

,nct_id,brief_title,brief_summary,detailed_description,overall_status,phase,study_type,criteria,gender,min_age,max_age,intervention,combined
533,NCT00000647,an open trial combining zidovudine interferon ...,to restore immunologic function and virus free...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,18 Years,60 Years,None,an open trial combining zidovudine interferon ...
534,NCT00000648,a pilot study evaluating penicillin g and ceft...,to provide information on the response of hiv ...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,13 Years,N/A,None,a pilot study evaluating penicillin g and ceft...
535,NCT00000646,pentoxifylline trental as a modulator of tumor...,to determine whether pentoxifylline lowers tum...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,18 Years,N/A,None,pentoxifylline trental as a modulator of tumor...
536,NCT00000649,an open label staggered rising dose cohort stu...,to assess the safety and tolerance of multiple...,None,Completed,None,Interventional,\n Inclusion Criteria\r\n\r\n Co...,All,18 Years,N/A,None,an open label staggered rising dose cohort stu...
537,NCT00000645,a phase i dose escalation study of synthetic h...,to determine the maximum tolerated dose mtd of...,None,Completed,None,Interventional,None,All,18 Years,N/A,None,a phase i dose escalation study of synthetic h...


In [26]:
import numpy as np

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
!pip install -q sentence-transformers

     |████████████████████████████████| 81kB 5.0MB/s 
     |████████████████████████████████| 2.3MB 40.1MB/s 
     |████████████████████████████████| 1.2MB 43.2MB/s 
     |████████████████████████████████| 901kB 36.3MB/s 
     |████████████████████████████████| 3.3MB 32.9MB/s 


In [30]:

from sentence_transformers import SentenceTransformer

In [31]:
corpus = list(ntc_1['combined'])

In [32]:
corpus

['novel medications for opiate detoxificationthe purpose of this study is to evaluate novel medications for opiate detoxification\n        Please contact site for information.\r\n      \n      The purpose of this study was to determine the effects of a metabolite of nicotine, cotinine,\r\n      on tobacco withdrawal symptoms. Cotinine has been shown to have psychoactive effects that are\r\n      similar as well as different from those of nicotine, however, little research has been\r\n      conducted examining the role cotinine plays in nicotine addiction. This study compared the\r\n      effects of cotinine with the nicotine patch, and a combination thereof on tobacco withdrawal\r\n      symptoms. The results showed that cotinine antagonizes the beneficial effects of the nicotine\r\n      patch in reducing withdrawal symptoms.\r\n    ClonidineN/A18 YearsAll',
 'mazindol for cocaine abusethe purpose of this study is to evaluate high and low dose mazindol for cocaine abuse\n        Pleas

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens/0_BERT were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentence_embeddings = model.encode(corpus)

In [ ]:
sentence_embeddings.shape

(538, 768)

In [ ]:
sentence_embeddings


array([[-0.5502616 ,  0.5686462 ,  0.31655106, ..., -0.19717577,
         0.26599857,  0.22266471],
       [-0.5167078 ,  0.9052687 , -0.53440255, ...,  0.12262804,
         0.1302084 ,  0.25650492],
       [-0.60170376,  0.51850796,  0.7413792 , ...,  0.0216009 ,
         0.07236539,  0.29971737],
       ...,
       [-0.28325987,  0.5885569 ,  0.09204266, ..., -0.20963895,
         0.05269158,  0.52780855],
       [-0.47259068,  0.62302154, -0.18179528, ..., -0.6421077 ,
        -0.10345159,  0.59282494],
       [-0.50625503,  0.6388311 , -0.42740387, ..., -0.5293951 ,
         0.19279128,  0.4706391 ]], dtype=float32)

In [ ]:
patient_details = "A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes."

In [ ]:
import numpy as np

In [61]:
ind = np.array(ntc_1['nct_id']).reshape(510,)

In [ ]:
trail_df = pd.DataFrame(sentence_embeddings, index = ind)

NameError: ignored

In [ ]:
trail_df

NameError: ignored

In [ ]:
patients = [patient_details]

In [ ]:
patient_embeddings = model.encode(patients)

In [ ]:
import scipy
from scipy.spatial import distance

In [ ]:
for query, patient_embedding in zip(patients, patient_embeddings):
    distances = scipy.spatial.distance.cdist([patient_embedding], sentence_embeddings, "cosine")[0]

In [ ]:
for query, patient_embedding in zip(patients, patient_embeddings):
    distances = scipy.spatial.distance.cdist([patient_embedding], sentence_embeddings, "cosine")[0]

In [ ]:
dis_df = pd.DataFrame(distances, index = ind)

In [ ]:
dis_df['Proximity'] = (1-dis_df.iloc[:,0])

In [ ]:
dis_df.sort_values('Proximity', ascending=False).index[0:9]

Index(['NCT00000221', 'NCT00000280', 'NCT00000543', 'NCT00000296',
       'NCT00000178', 'NCT00000199', 'NCT00000474', 'NCT00000448',
       'NCT00000160'],
      dtype='object')

In [ ]:
results = zip(range(len(distances)), distances)
results = sorted(results, key=lambda x: x[1])

In [ ]:
print("\n\n======================\n\n")
print("Patient:", query)
print("\nTop 10 most similar trials in corpus:")





Patient: A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes.

Top 10 most similar trials in corpus:


In [ ]:
for idx, distance in results[0:10]:
    print(corpus[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

alternate day buprenorphine administration phase ithe purpose of this study is to determine if two times a subject s daily maintenance dose will hold for hours without changes in agonist and antagonist effects please contact site for information next to osteoarthritis fms is the most common arthritis related disease symptoms include musculoskeletal pain fatigue headaches irritable bowel syndrome morning stiffness and sleep disturbances fluctuating severity pain and frustration with the difficulty of diagnosis and treatment lead patients to continually seek help from health care professionals there is no known cause or cure for this disease this study tests the effects of social support and education on the health and well being of people with fibromyalgia fms we recruited adults with a confirmed diagnosis of fms from a large health maintenance organization to be eligible people had to meet the american college of rheumatology guidelines for fms after we confirmed the diagnosis we assig

In [ ]:
model = ClinicalBertSimilarity()

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Bio-BERT


In [33]:
!wget https://setup.johnsnowlabs.com/nlu/colab.sh -O - | bash

--2021-06-08 19:43:05--  https://setup.johnsnowlabs.com/nlu/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/nlu/master/scripts/colab_setup.sh [following]
--2021-06-08 19:43:06--  https://raw.githubusercontent.com/JohnSnowLabs/nlu/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1662 (1.6K) [text/plain]
Saving to: ‘STDOUT’

Installing  NLU 3.0.1 with  PySpark 3.0.2 and Spark NLP 3.0.1 for Google Colab ...
-                   100%[===================>]   1.62K  --.-KB/s    in 0s      

In [34]:
! pip install nlu pyspark==3.0.1

     |████████████████████████████████| 204.2MB 79kB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=8065f5084031bfc86bc8b439c87a4801c4ba85407b225ddf2272f3ef8f9724d1
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
  Found existing installation: pyspark 3.0.2
    Uninstalling pyspark-3.0.2:
      Successfully uninstalled pyspark-3.0.2


In [36]:
import nlu

In [37]:
import os
! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [ ]:
corpus[0]

'the berkeley orthokeratology studyto evaluate the relative efficacy of orthokeratology primarily by assessment of changes in central corneal thickness astigmatism visual acuity endothelial cell density and corneal curvature to evaluate the relative safety of orthokeratology primarily by assessment of changes in central corneal thickness astigmatism visual acuity endothelial cell density induced corneal edema and epithelial staining to assess the duration of any orthokeratology treatment effect to study the mechanisms by which refractive error and visual acuity changes occur in particular the contribution that comes from changes in corneal curvature and shape to determine whether there were any predisposing ocular factors that could be used to predict which subjects will experience changes or complications myopic volunteers ages to who had not worn contact lenses were eligible to participate in the study if they were free of ocular disease were in good physical health and were not taki

In [ ]:
df_embed_2 =nlu.load('biobert').predict(corpus[1]) 

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [ ]:
import numpy as np

In [68]:
text =corpus[0:len(corpus)]
bioembeddings_df = nlu.load('en.embed_sentence.biobert.clinical_base_cased').predict(text, output_level='document')
bioembeddings_df

sent_biobert_clinical_base_cased download started this may take some time.
Approximate size to download 386.6 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


,document,text,sentence,origin_index,sentence_embedding_biobert
0,novel medications for opiate detoxificationthe...,novel medications for opiate detoxificationthe...,[novel medications for opiate detoxificationth...,0,"[-0.01818219944834709, -0.15203282237052917, -..."
1,mazindol for cocaine abusethe purpose of this ...,mazindol for cocaine abusethe purpose of this ...,[mazindol for cocaine abusethe purpose of this...,1,"[-0.09215410053730011, -0.1510278880596161, -0..."
2,role of metabolites in nicotine dependencethe ...,role of metabolites in nicotine dependencethe ...,[role of metabolites in nicotine dependencethe...,2,"[-0.07858985662460327, -0.2618032693862915, -0..."
3,pharmacotherapy for schizophrenic drug usersth...,pharmacotherapy for schizophrenic drug usersth...,[pharmacotherapy for schizophrenic drug userst...,3,"[-0.03150765225291252, -0.12501700222492218, 0..."
4,effects of phenytoin on cocaine use in humanst...,effects of phenytoin on cocaine use in humanst...,[effects of phenytoin on cocaine use in humans...,4,"[-0.06603291630744934, -0.25264185667037964, -..."
...,...,...,...,...,...
505,multi center trial of anti thymocyte globulin ...,multi center trial of anti thymocyte globulin ...,[multi center trial of anti thymocyte globulin...,8589934842,"[-0.11909516155719757, -0.05130807310342789, -..."
506,evaluation of subcutaneous desferrioxamine as ...,evaluation of subcutaneous desferrioxamine as ...,[evaluation of subcutaneous desferrioxamine as...,8589934843,"[-0.02875000797212124, -0.06405619531869888, -..."
507,viral activation transfusion study vatsthe pur...,viral activation transfusion study vatsthe pur...,[viral activation transfusion study vatsthe pu...,8589934844,"[-0.07578212767839432, -0.0812942236661911, -0..."
508,nhlbi type ii coronary intervention studyto de...,nhlbi type ii coronary intervention studyto de...,[nhlbi type ii coronary intervention studyto d...,8589934845,"[0.15682768821716309, -0.15219327807426453, -0..."


In [39]:
bio_corpus = np.zeros((1,768))

In [40]:
bio_corpus= np.transpose(bioembeddings_df.iloc[0,4].reshape(768,1))

In [41]:
bio_corpus.shape

(1, 768)

In [42]:
for i in range(1,len(corpus)):
  next = np.transpose(bioembeddings_df.iloc[i,4].reshape(768,1))
  bio_corpus = np.vstack((bio_corpus,next))


In [ ]:
#next = np.transpose(embeddings_df.iloc[2,4].reshape(768,1))

In [ ]:
#new = np.vstack((final,next))

In [43]:
bio_corpus.shape

(510, 768)

In [44]:
#new

In [77]:
patient_details = " physician is called to see a 67-year-old woman who underwent cardiac catheterization via the right femoral artery earlier in the morning. She is now complaining of a cool right foot. Upon examination she has a pulsatile mass in her right groin with loss of distal pulses, and auscultation reveals a bruit over the point at which the right femoral artery was entered."

In [47]:
patient_details_p = preprocess(patient_details)

In [ ]:
patient_details

[['A 58-year-old African-American woman presents to the ER with episodic pressing']]

In [66]:

patient_embed =  nlu.load('en.embed_sentence.biobert.clinical_base_cased').predict(patient_details, output_level='document')

sent_biobert_clinical_base_cased download started this may take some time.
Approximate size to download 386.6 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [49]:
patient_embed

,document,text,sentence,origin_index,sentence_embedding_biobert
0,"A 75F with a PMHx significant for severe PVD, ...","A 75F with a PMHx significant for severe PVD, ...","[A 75F with a PMHx significant for severe PVD,...",8589934592,"[-0.1448669731616974, 0.1046062484383583, 0.07..."


In [50]:
patient_embed.iloc[0,4].shape

(768,)

In [51]:
patient_bioembed = np.transpose(patient_embed.iloc[0,4].reshape(768,1))

In [52]:
patient_bioembed.shape

(1, 768)

In [56]:
import scipy
from scipy.spatial import distance

In [57]:
distances = scipy.spatial.distance.cdist(patient_bioembed,bio_corpus, "cosine")[0]

In [58]:
distances.min()

0.0809289864764825

In [62]:
dis_df = pd.DataFrame(distances, index = ind)

In [63]:
dis_df

,0
NCT00000279,0.100763
NCT00000277,0.113691
NCT00000284,0.116953
NCT00000281,0.101451
NCT00000285,0.095652
...,...
NCT00000597,0.110369
NCT00000595,0.102808
NCT00000593,0.116934
NCT00000594,0.124848


In [64]:
dis_df['Proximity'] = (1-dis_df.iloc[:,0])

In [65]:
dis_df.sort_values('Proximity', ascending=False).index[0:9]

Index(['NCT00000217', 'NCT00000242', 'NCT00000292', 'NCT00000526',
       'NCT00000190', 'NCT00000332', 'NCT00000525', 'NCT00000274',
       'NCT00000351'],
      dtype='object')

In [74]:
distance_table = pd.DataFrame(dis_df['Proximity'])

In [76]:
distance_table

,Proximity
NCT00000279,0.899237
NCT00000277,0.886309
NCT00000284,0.883047
NCT00000281,0.898549
NCT00000285,0.904348
...,...
NCT00000597,0.889631
NCT00000595,0.897192
NCT00000593,0.883066
NCT00000594,0.875152
